## 台北市房價分析
   
   # 資料預處理

   檢查資料是否有重複的可能性以及遺失值

   坪數與房價價格轉換

       1、 單價(萬/坪): 單價元平方公尺乘上3.305並除10000
   
       2、 (建物、土地、主建物、附屬建物、陽台面積、車位)的坪數 (3.305平方公尺) 對除

       3、 (房屋土地及車位)的總價元除上10000,得到萬元為單位的價格

   屋齡資料上的轉換

       1、 轉換資料到適合計算屋齡的時間資料格式

       2、 交易資料跟建築完成年月日資料對減獲得屋齡

       3、 對屋齡除上 365 天
       
       4、 屋齡為負則化為0 ,在另新變數為是否有購買預售屋(1 : 代表有購買預售屋 , 0 : 未購買)

   
     轉換 轉移樓層
          
          1、總樓層資料換成數字 

          2、 找出轉移層次中的騎樓、夾層、地下室、平台、多樓層等交易細節，轉為屬質變數

          3、轉移層次中的全轉為透天厝，其餘轉為數字

     計算面積占比 包含(主建物、附屬建物、陽台面積以及公占比)

     轉換備註資料(包含是否為特殊關係交易、違章建物、未登記建物以及公共設施)





In [320]:
import pandas as pd
data113 = pd.read_excel(r'C:\資料探勘\經緯度資料標記\113_label.xlsx',index_col=False)


## 確認資料是否有重複


In [321]:
print(f'資料重複筆數: {data113[data113.duplicated()].shape[0]}')

資料重複筆數: 0


In [322]:
# REMOVE DUPLICATE DATA SET 

data113= data113[- data113.duplicated()]

## 檢查資料是否有重複的可能以及遺失值

解釋 : 
       1、 非都市土地用地資料只有一筆資料，占比很小，因此列為不考慮的變數

       2、 都市土地使用分區遺失資料數為 563 ,推測可能為是沒有土地的房屋

       3、 建築完成年月日遺失值，推測可能是因為房子在內政部開始登記前建造，所以有部分資料蒐集不到

       4、 總樓層、移轉層次推測是為只有土地跟車位交易

       5、 單價元平方公尺 : 推測可能是車位

In [323]:
print(f'資料列數 : {data113.shape[0]}' ,f'資料行數 : {data113.shape[1]}')
print('=======================================')

print('資料遺失情況')
print(data113.isna().sum())

資料列數 : 12572 資料行數 : 37
資料遺失情況
鄉鎮市區               0
交易標的               0
土地位置建物門牌           0
土地移轉總面積平方公尺        0
都市土地使用分區         277
非都市土地使用分區      12572
非都市土地使用編定      12572
交易年月日              0
交易筆棟數              0
移轉層次             115
總樓層數             127
建物型態               0
主要用途             481
主要建材             116
建築完成年月           539
建物移轉總面積平方公尺        0
建物現況格局.房           0
建物現況格局.廳           0
建物現況格局.衛           0
建物現況格局.隔間          0
有無管理組織             0
總價元                0
單價元平方公尺          672
車位類別            8707
車位移轉總面積平方公尺        0
車位總價元              0
備註              3860
編號                 0
主建物面積              0
附屬建物面積             0
陽台面積               0
電梯                 0
移轉編號            8009
m                  0
n                  0
經度               193
緯度               193
dtype: int64


. 調查移轉層數遺失原因

In [324]:
print(f'移轉層次遺失次數 : {data113[data113['移轉層次'].isna()].shape}')

print(f'交易標的只有土地及車位以及移轉層次的遺失筆數 : {data113.query(('((交易標的 == "車位") | (交易標的 == "土地")) & @pd.isna(移轉層次)')).shape[0]}')

移轉層次遺失次數 : (115, 37)
交易標的只有土地及車位以及移轉層次的遺失筆數 : 115


In [325]:
print(data113['交易標的'].value_counts())

交易標的
房地(土地+建物)       8597
房地(土地+建物)+車位    3187
車位               672
建物               116
Name: count, dtype: int64


## 原因是建物含有遺失值導致

In [326]:
pd.set_option("display.max_columns",None)
print(data113.query('(交易標的 == "建物") & @pd.isna(移轉層次)'))

Empty DataFrame
Columns: [鄉鎮市區, 交易標的, 土地位置建物門牌, 土地移轉總面積平方公尺, 都市土地使用分區, 非都市土地使用分區, 非都市土地使用編定, 交易年月日, 交易筆棟數, 移轉層次, 總樓層數, 建物型態, 主要用途, 主要建材, 建築完成年月, 建物移轉總面積平方公尺, 建物現況格局.房, 建物現況格局.廳, 建物現況格局.衛, 建物現況格局.隔間, 有無管理組織, 總價元, 單價元平方公尺, 車位類別, 車位移轉總面積平方公尺, 車位總價元, 備註, 編號, 主建物面積, 附屬建物面積, 陽台面積, 電梯, 移轉編號, m, n, 經度, 緯度]
Index: []


# 對平方公尺單價元遺失原因是為車位價格分開而因此無法計算

In [327]:
print(f"平方公尺單價元遺失次數 : {data113['單價元平方公尺'].isna().sum()} ")
print(f'平方公尺單價元跟交易標的為車位遺失次數 : {data113.query(('(交易標的 == "車位")  & @pd.isna(單價元平方公尺)')).shape[0]}')


平方公尺單價元遺失次數 : 672 
平方公尺單價元跟交易標的為車位遺失次數 : 672


## 坪數與房價價格轉換

In [328]:
data113['單價(萬/坪)'] = data113['單價元平方公尺']*3.305/10000

data113['土地移轉總面積(坪)'] = data113['土地移轉總面積平方公尺']/3.305
data113['建物移轉總面積(坪)'] = data113['建物移轉總面積平方公尺']/3.305
data113['車位移轉總面積(坪)'] = data113['車位移轉總面積平方公尺']/3.305
data113['主建物面積(坪)'] = data113['主建物面積']/3.305
data113['附屬建物面積(坪)'] = data113['附屬建物面積']/3.305
data113['陽台面積(坪)'] = data113['陽台面積']/3.305

data113['車位總價元(萬)'] = data113['車位總價元']/10000
data113['總價元(萬)'] = data113['總價元']/10000




## 計算屋齡
   
    1、轉換資料到適合計算屋齡的時間資料格式

    2、交易資料跟建築完成年月日資料對減獲得屋齡

    3、對屋齡除上 365 天

    4、屋齡為負則化為0 ,在另新變數為是否有購買預售屋(1 : 代表有購買預售屋 , 0 : 未購買)

    

In [329]:
import math

def transfor_date(date_str):
    try:

        date_str = str(int(float(date_str)))
        date = int(date_str[-2:])
        month = int(date_str[-4:-2])
        year = int(date_str[:-4])+1911
        DATE = pd.to_datetime(f"{year}-{month}-{date}",format='%Y-%m-%d',errors='coerce')

        return DATE.date()
    
    except ValueError:
        return math.nan


In [330]:
data113['交易年月日'] = data113['交易年月日'].apply(lambda x : transfor_date(str(x)) )

In [331]:
print(data113.head())

  鄉鎮市區          交易標的                       土地位置建物門牌  土地移轉總面積平方公尺  \
0  士林區            車位  臺北市士林區芝玉路一段２２２、２２６、２２６之１號地下一層         4.99   
1  士林區            車位         臺北市士林區中山北路七段３８巷１１號地下三層         0.53   
2  士林區     房地(土地+建物)             臺北市士林區延平北路九段２１３號三樓        42.67   
3  士林區  房地(土地+建物)+車位             臺北市士林區天母西路５０巷２５號五樓        54.19   
4  士林區     房地(土地+建物)                  臺北市士林區社中街１２０號        18.00   

                               都市土地使用分區  非都市土地使用分區  非都市土地使用編定       交易年月日  \
0                                     住        NaN        NaN  2024-03-01   
1                                     住        NaN        NaN  2024-02-19   
2  都市：其他:遊樂區(細部計畫尚未完成，尚未能准許依變更後計畫用途使用)。        NaN        NaN  2024-02-06   
3                                     住        NaN        NaN  2024-02-15   
4                                     住        NaN        NaN  2024-02-17   

       交易筆棟數   移轉層次 總樓層數             建物型態  主要用途      主要建材    建築完成年月  \
0  土地1建物1車位1   地下一層  十八層               其他  停車空間  鋼骨鋼筋混凝土造

In [332]:
import math

data113['建築完成年月']= data113['建築完成年月'].apply(lambda x: transfor_date(str(x)))

In [333]:
data113['交易年月日']=pd.to_datetime(data113['交易年月日'], errors='coerce')
data113['建築完成年月']=pd.to_datetime(data113['建築完成年月'], errors='coerce')



In [334]:
data113['屋齡'] = (data113['交易年月日']-data113['建築完成年月']).dt.days


In [335]:
# Replace negative values with 0

data113['屋齡'] = data113['屋齡'].apply(lambda x: max(x, 0))


In [336]:
data113['屋齡'] = data113['屋齡']/365

In [337]:
data113['是否為預購屋'] = data113['屋齡'].apply(lambda x : '1' if x==0 else '0' )

In [338]:
import re
import matplotlib.pyplot as plt

In [339]:
data113.head()

,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,總樓層數,建物型態,主要用途,主要建材,建築完成年月,建物移轉總面積平方公尺,建物現況格局.房,建物現況格局.廳,建物現況格局.衛,建物現況格局.隔間,有無管理組織,總價元,單價元平方公尺,車位類別,車位移轉總面積平方公尺,車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號,m,n,經度,緯度,單價(萬/坪),土地移轉總面積(坪),建物移轉總面積(坪),車位移轉總面積(坪),主建物面積(坪),附屬建物面積(坪),陽台面積(坪),車位總價元(萬),總價元(萬),屋齡,是否為預購屋
0,士林區,車位,臺北市士林區芝玉路一段２２２、２２６、２２６之１號地下一層,4.99,住,NaN,NaN,2024-03-01,土地1建物1車位1,地下一層,十八層,其他,停車空間,鋼骨鋼筋混凝土造,2008-11-12,17.32,0,0,0,無,有,3300000,NaN,坡道平面,17.31,3300000,僅車位交易；,RPPNMLQJLHJGFEA67EA,0.00,0.00,0.00,有,2.0,113,3,25.109462,121.534093,NaN,1.509834,5.240545,5.237519,0.000000,0.000000,0.000000,330.0,330.0,15.309589,0
1,士林區,車位,臺北市士林區中山北路七段３８巷１１號地下三層,0.53,住,NaN,NaN,2024-02-19,土地1建物1車位1,地下三層,十二層,其他,停車空間,鋼筋混凝土造,2008-09-04,44.93,0,0,0,無,有,3150000,NaN,坡道平面,44.93,3150000,僅車位交易；,RPPOMLLLKHJGFEA27DA,0.00,0.00,0.00,有,38.0,113,2,25.106497,121.527479,NaN,0.160363,13.594554,13.594554,0.000000,0.000000,0.000000,315.0,315.0,15.468493,0
2,士林區,房地(土地+建物),臺北市士林區延平北路九段２１３號三樓,42.67,都市：其他:遊樂區(細部計畫尚未完成，尚未能准許依變更後計畫用途使用)。,NaN,NaN,2024-02-06,土地1建物1車位0,三層，陽臺,三層,公寓(5樓含以下無電梯),住家用,鋼筋混凝土造,1976-02-14,71.63,6,1,1,有,無,10000000,139606.0,NaN,0.00,0,頂樓加蓋；,RPPOMLPJKHJGFEA47DA,71.63,0.00,0.00,無,NaN,113,2,25.098656,121.546989,46.139783,12.910741,21.673222,0.000000,21.673222,0.000000,0.000000,0.0,1000.0,48.010959,0
3,士林區,房地(土地+建物)+車位,臺北市士林區天母西路５０巷２５號五樓,54.19,住,NaN,NaN,2024-02-15,土地1建物1車位2,五層,十四層,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,2001-04-23,304.02,4,2,3,有,有,68000000,223669.0,坡道平面,0.00,6000000,陽台外推；,RPRNMLOJLHJGFEA57DA,139.76,2.09,15.91,有,NaN,113,2,25.092872,121.506457,73.922605,16.396369,91.987897,0.000000,42.287443,0.632375,4.813918,600.0,6800.0,22.830137,0
4,士林區,房地(土地+建物),臺北市士林區社中街１２０號,18.00,住,NaN,NaN,2024-02-17,土地1建物1車位0,一層,四層,公寓(5樓含以下無電梯),住商用,鋼筋混凝土造,1977-05-27,48.21,0,0,0,無,無,4490000,93134.0,NaN,0.00,0,親友、員工、共有人或其他特殊關係間之交易；,RPRNMLPJLHJGFEA67DA,44.27,3.94,0.00,無,NaN,113,2,25.137294,121.547139,30.780787,5.446293,14.586989,0.000000,13.394856,1.192133,0.000000,0.0,449.0,46.758904,0


## 轉換交易層數資料

   1、 轉換購買總樓層數為數量

   2、轉換移轉層數資料，並且觀察是否有買多層樓的房屋，若有則會轉為是否購買一層以上的房屋
   
   已知購買全棟是為透天厝 


In [340]:
from chinesenumber import NumberParser
parser = NumberParser()

總樓層數移轉

In [341]:
data113['總樓層數'] = data113['總樓層數'].apply(lambda  x : 
                                  float(parser.numberify(str(x).split('層')[0])))

In [342]:
data113['移轉層次'].head(50)

0      地下一層
1      地下三層
2     三層，陽臺
3        五層
4        一層
5        四層
6        三層
7        一層
8        一層
9        四層
10       六層
11       五層
12      地下層
13      十二層
14       五層
15      三十層
16      NaN
17       一層
18      地下層
19       三層
20       五層
21       四層
22       一層
23       三層
24       二層
25       四層
26       一層
27       二層
28       四層
29       三層
30      地下層
31      地下層
32       十層
33       三層
34    四層，陽臺
35       二層
36      十二層
37       六層
38       四層
39       五層
40       一層
41     地下三層
42       十層
43       六層
44       三層
45       五層
46     地下二層
47       六層
48      地下層
49       一層
Name: 移轉層次, dtype: object

In [343]:
import re

def identify_mutiple_floors(x):

    # Convert input to string for processing
    x = str(x)

    x = parser.numberify(x)
    x = x.split('層')

    x = [re.sub(r'\D', '', word) for word in x]
    x = [word for word in x if word != '']

    

    return len(x)+1


In [344]:
floor = data113['移轉層次'].apply(lambda x : identify_mutiple_floors(x))
max_floor = data113['移轉層次'].apply(lambda x : identify_mutiple_floors(x)).idxmax()

print(data113['移轉層次'].apply(lambda x : identify_mutiple_floors(x)).max())

data113['移轉層次'].iloc[max_floor]

8


'二層，三層，四層，五層，六層，七層，八層'

In [345]:

# set definition fuinction for 

def numeric_transfor(floor,number):
    
    try:
        
        
        
        if floor == '地下層':
            
            return  -1
        
        floor = str(floor).split('層')[number -1 ]   
        
        if floor == '全':
            
            return 1
         
             
        
        # data.split('層') => ['二層', ] , the length of list is 2 
                     
        if re.search(r'地下',floor):
            x = floor.replace(r'地下','')
            x = x.replace(r'，','')
            x = - int(parser.numberify(x))
            return x
        
        else:
            x = floor.replace(r'，','')
            x = int(parser.numberify(x))
            return x
        

    except Exception as e :
        
        return floor


In [346]:
pd.set_option("display.max_rows",None)
data113['移轉層次'].apply(lambda x : str(x).split('層'))

0                                    [地下一, ]
1                                    [地下三, ]
2                                   [三, ，陽臺]
3                                      [五, ]
4                                      [一, ]
5                                      [四, ]
6                                      [三, ]
7                                      [一, ]
8                                      [一, ]
9                                      [四, ]
10                                     [六, ]
11                                     [五, ]
12                                    [地下, ]
13                                    [十二, ]
14                                     [五, ]
15                                    [三十, ]
16                                     [nan]
17                                     [一, ]
18                                    [地下, ]
19                                     [三, ]
20                                     [五, ]
21                                     [四, ]
22        

In [347]:
data113['第一筆樓層數'] = data113['移轉層次'].apply(lambda x : numeric_transfor(x,1))

In [348]:
data113['移轉層次']

0                               地下一層
1                               地下三層
2                              三層，陽臺
3                                 五層
4                                 一層
5                                 四層
6                                 三層
7                                 一層
8                                 一層
9                                 四層
10                                六層
11                                五層
12                               地下層
13                               十二層
14                                五層
15                               三十層
16                               NaN
17                                一層
18                               地下層
19                                三層
20                                五層
21                                四層
22                                一層
23                                三層
24                                二層
25                                四層
26                                一層
2

In [349]:
pd.set_option("display.max_rows",None)
data113['第一筆樓層數']

0                   -1
1                   -3
2                    3
3                    5
4                    1
5                    4
6                    3
7                    1
8                    1
9                    4
10                   6
11                   5
12                  -1
13                  12
14                   5
15                  30
16                 nan
17                   1
18                  -1
19                   3
20                   5
21                   4
22                   1
23                   3
24                   2
25                   4
26                   1
27                   2
28                   4
29                   3
30                  -1
31                  -1
32                  10
33                   3
34                   4
35                   2
36                  12
37                   6
38                   4
39                   5
40                   1
41                  -3
42                  10
43         

In [350]:
data113['第二筆層數'] = data113['移轉層次'].apply(lambda x : numeric_transfor(x,2))


# Count empty rows (where 'Content' is empty or whitespace)
empty_rows = data113[data113['第二筆層數'].str.strip() == ""].shape[0]

print(empty_rows/data113.shape[0])

0.8576996500159084


In [351]:
data113['第二筆層數']

0                     
1                     
2                  ，陽臺
3                     
4                     
5                     
6                     
7                     
8                     
9                     
10                    
11                    
12                  -1
13                    
14                    
15                    
16                 NaN
17                    
18                  -1
19                    
20                    
21                    
22                    
23                    
24                    
25                    
26                    
27                    
28                    
29                    
30                  -1
31                  -1
32                    
33                    
34                 ，陽臺
35                    
36                    
37                    
38                    
39                    
40                    
41                    
42                    
43         

計算移轉層數為(全)的建物型態

In [352]:
print(f'移轉層數為全的層數筆數 : {data113.query('(移轉層次 == "全")').shape[0]}')
print(f'移轉層數為全以及建物型態為透天厝交集筆數 : {data113.query('(移轉層次 == "全") & (建物型態 == "透天厝")').shape[0]}')

移轉層數為全的層數筆數 : 379
移轉層數為全以及建物型態為透天厝交集筆數 : 379


# 定義是否購買多層數

In [353]:
import re

def identify_mutiple_floors(x):

    # Convert input to string for processing
    x = str(x)

    x = parser.numberify(x)
    x = x.split('層')

    x = [re.sub(r'\D', '', word) for word in x]
    x = [word for word in x if word != '']

    

    if len(x) > 1 :  
        return '1'  
    else:
        return '0'  


In [354]:
data113['是否夠買一層以上的房子']= data113['移轉層次'].apply(lambda x:identify_mutiple_floors(x))  

## 是否購買夾層屋

In [355]:
data113['購買是否有買夾層屋'] = data113['移轉層次'].apply(lambda x : '1' 
                                       if isinstance(x,str) and re.search(r'夾層',x) else '0')

## 公設面積包含電梯或樓梯

In [356]:
data113['公設面積包含電梯或樓梯'] = data113['移轉層次'].apply(lambda x : '1' 
                                         if isinstance(x,str) and re.search(r'電梯樓梯',x) else '0')

## 公設面積包含屋頂突出物

In [357]:
data113['公設面積包含屋頂突出物'] = data113['移轉層次'].apply(lambda x : '1'
                                          if isinstance(x,str) and re.search(r'屋頂突出物',x) else '0')

## 附屬面積含平台

In [358]:
data113['附屬面積含平台'] = data113['移轉層次'].apply(lambda x : '1' 
                                     if isinstance(x,str) and re.search(r'平台',x) else '0')

## 附屬面積含露台

In [359]:
data113['附屬面積含露台'] = data113['移轉層次'].apply(lambda x : '1' 
                                     if isinstance(x,str) and re.search(r'露台',x) else '0')

## 購買時是否有騎樓

In [360]:
data113['購買時是否有騎樓'] = data113['移轉層次'].apply(lambda x : '1' 
                                      if isinstance(x,str) and re.search(r'騎樓',x) else '0')

## 是否有購地下樓層

In [361]:
data113['是否有購地下樓層'] = data113['移轉層次'].apply(lambda x : '1' 
                                      if isinstance(x,str) and re.search(r'地下',x) else '0')

## 主要用途\

In [362]:
pd.set_option("display.max_row",None)

print(data113['主要用途'].value_counts())

主要用途
住家用                                                             9315
商業用                                                             1437
辦公用                                                              346
其他                                                               248
停車空間                                                             161
工業用                                                              151
見使用執照                                                            113
住商用                                                              110
一般事務所                                                             58
一般零售業甲組                                                           20
（Ｇ２）一般事務所（第２８組）                                                   18
防空避難室兼停車場                                                         18
停車場                                                               11
一般零售業                                                             11
一般零售業（甲組）                    

In [363]:
print(data113['交易標的'].value_counts())

交易標的
房地(土地+建物)       8597
房地(土地+建物)+車位    3187
車位               672
建物               116
Name: count, dtype: int64


In [364]:
print(f"房地主要用途遺失值:{data113.query("(交易標的 == '房地(土地+建物)')")['主要用途'].isna().sum()}")
print(f"房地加車位主要用途遺失值:{data113.query("(交易標的 == '房地(土地+建物)+車位')")['主要用途'].isna().sum()}")
print(f"車位主要用途遺失值:{data113.query("(交易標的 == '車位')")['主要用途'].isna().sum()}")
print(f"車位主要用途遺失值:{data113.query("(交易標的 == '建物')")['主要用途'].isna().sum()}")
print(f"資料主要用途遺失值:{data113['主要用途'].isna().sum()}")

房地主要用途遺失值:354
房地加車位主要用途遺失值:2
車位主要用途遺失值:116
車位主要用途遺失值:9
資料主要用途遺失值:481


In [365]:
def transfor_purpose(x):

    if isinstance(x,str):

        if re.search(r"零售",x) and re.search(r"停車",x):

            return '零售用途及停車用途'

        elif re.search(r"零售",x) :

            return '零售用途'

        elif re.search(r"停車",x):

            return '停車用途'

        elif re.search(r'事務',x):
            return '事務所'

        else :
            return x 
    else:
        return x

In [366]:
data113['主要用途'] = data113['主要用途'].apply(lambda x : transfor_purpose(x))

In [367]:
print(data113['主要用途'].value_counts())

主要用途
住家用                9315
商業用                1437
辦公用                 346
其他                  248
停車用途                205
工業用                 151
見使用執照               113
住商用                 110
事務所                  80
零售用途                 57
住工用                   5
多戶住宅                  5
集合住宅                  4
日常服務業                 4
策略性產業（資訊服務業）          2
日常服務業〈不含理髮及美容業〉       2
集合住宅（Ｈ２）              1
防空避難室                 1
社區遊憩設施                1
策略性產業〈２８組資訊服務業〉       1
戶內遊憩設施                1
農業用                   1
工商用                   1
Name: count, dtype: int64


## 都市土地使用轉換 對公共設施用地

In [368]:
def transfor_urban(x):

    if  isinstance(x,str):

        if re.search("公共設施用地",x) :

            return '公共設施用途'

        else :

            return x

    else :

        return x

In [369]:
data113['都市土地使用分區'] = data113['都市土地使用分區'].apply(lambda x : transfor_urban(x))

In [370]:
data113['都市土地使用分區'].value_counts()

都市土地使用分區
住                                                                               7085
商                                                                               4017
工                                                                                242
都市：其他:第三種住宅區。                                                                    204
都市：其他:住宅用地(民生社區特定專用區)。                                                            95
公共設施用途                                                                            91
都市：其他:特定業務區(信義計畫特定專用區)。                                                           67
都市：其他:特定觀光商業專用區。                                                                  62
都市：其他:特定休閒旅館住宅專用區。                                                                42
都市：其他:第二種住宅區。                                                                     32
都市：其他:聯合開發區(捷)。                                                                   29
都市：其他:特定專用區(供一般商業使用)。                                   

## 轉換土地比例 (主建物占比、附屬建物占比、車位占比、公設比)

  . 建物移轉面積算進主建物面積、車位面積、陽台面積、附屬面積以及公設面積

  . 主建物占比 = (主建物面積)/(建物移轉面積 )

  . 陽台占比 = 陽台面積/(建物移轉總面積)

  . 附屬建物占比 = 附屬建物面積/(建物移轉總面積 )

  . 車位佔比 = 車位面積 / 建物移轉面積

  . 公占比 = 1 - 主建物占比 - 陽台面積占比 - 附屬面積占比  https://labhouse.ebc.net.tw/flash/2329

  . 是否有把 陽台 、附屬建物面積 跟 公占比 列入 坪數價格 : (總價格 - 車位價格)/ (建物移轉總面積) = 每坪價格 ， 相等答 1

  . 附屬建物價格及公設可不列入實際價錢坪數及總價錢裡，因此若主建物占比接近100%則列為附屬建物跟公設不列入坪數範圍內
     https://www.moi.gov.tw/News_Content.aspx?n=2&s=13530#:~:text=%E4%BE%8B%E5%A6%82%EF%BC%8C%E4%BB%A5%E5%90%AB%E8%BB%8A%E4%BD%8D%E4%BA%A4,%E6%A0%BC%E8%B3%87%E8%A8%8A%E6%9B%B4%E5%8A%A0%E9%80%8F%E6%98%8E%E3%80%82



## 主建物比例

In [371]:
data113['主建物占比'] = data113['主建物面積(坪)']/data113['建物移轉總面積(坪)']

## 陽台面積占比

In [372]:
data113['陽台面積占比'] = data113['陽台面積(坪)']/data113['建物移轉總面積(坪)'] 

## 附屬面積占比 

In [373]:
data113['附屬面積占比'] = data113['附屬建物面積(坪)']/data113['建物移轉總面積(坪)'] 

## 公占比

In [374]:
data113['公佔比'] = 1 - data113['主建物占比'] - data113['陽台面積占比'] - data113['附屬面積占比']

## 附屬建物是否記在坪數價格裡面
   
  . 車位若沒有面積只有車位總價，那麼車位坪數價格則會被列為房屋實價價格   https://mlvr.land.moi.gov.tw/jsp/qa.jsp
  
  . 方法 : (總價 - 車位價格) / (建物轉移面積) 是否 小於 坪數價格，如果是的話則列為附屬建物或公設比不計坪數價格

  . 有記附屬面積為面積計 1 ,無記附屬面積為 0
    

In [375]:
def record_price_on_adscript_land(car_total_land,car_totel_price,building_total_price,builing_total_land,land_price):

    try:

        if car_total_land >= 0 and  car_totel_price >= 0:           

            return int((((building_total_price - car_totel_price )/
                         (builing_total_land)) >= land_price))

    except Exception as e:
        return '未擁有這些資料'




In [376]:

data113['是否有記附屬面積'] = data113.apply(lambda row : record_price_on_adscript_land(
row['車位總價元(萬)'],
row['車位移轉總面積(坪)'],
row['總價元(萬)'],
row['建物移轉總面積(坪)'],
row['單價(萬/坪)']
),axis=1)

   

In [377]:
data113['備註'].head()

0                   僅車位交易；
1                   僅車位交易；
2                    頂樓加蓋；
3                    陽台外推；
4    親友、員工、共有人或其他特殊關係間之交易；
Name: 備註, dtype: object

## 轉換 備註 資料
   
   . 違章建物法規 : https://law.moj.gov.tw/LawClass/LawAll.aspx?pcode=D0070124

   . 違建定義 : https://masterlaw.com.tw/%e9%81%95%e5%bb%ba%e4%b8%80%e5%ae%9a%e8%a6%81%e6%8b%86%e5%97%8e/ 、 https://www.hbhousing.com.tw/News/Detail.aspx?Num=5143


In [378]:
import re

# design "未登記建物" 、 公共設施 、增建、頂樓加蓋 、陽台外推以及增建 

def transforNOTE(text):
    
    if pd.isna(text):
        
        return '一般交易'
    
    if isinstance(text,str) and re.search(r"未登記建物",text) and re.search(r"公共設施",text) and re.search(r"特殊關係間之交易",text) and (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)) :
        return "透過特殊關係的交易、有未登記建物、公共設施以及違章建物"    


    elif  isinstance(text,str) and re.search(r"特殊關係間之交易",text) and re.search(r"未登記建物",text) and re.search(r"公共設施",text) and not (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "透過特殊關係的交易、未登記建物及公共設施"
    
    elif  isinstance(text,str) and re.search(r"特殊關係間之交易",text) and re.search(r"未登記建物",text) and not re.search(r"公共設施",text) and  (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "透過特殊關係的交易、未登記建物及違章建築"
    
    elif isinstance(text,str) and not re.search(r"未登記建物",text) and re.search(r"特殊關係間之交易",text) and  re.search(r"公共設施",text) and  (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "透過特殊關係的交易、公共設施及違章建物"
    
    elif isinstance(text,str) and  re.search(r"未登記建物",text) and not re.search(r"特殊關係間之交易",text) and  re.search(r"公共設施",text) and  (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "未登記建物、公共設施及違章建物"


    elif isinstance(text,str) and not re.search(r"未登記建物",text) and not re.search(r"特殊關係間之交易",text) and  re.search(r"公共設施",text) and  (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "公共設施及違章建物"
    
    elif isinstance(text,str) and not re.search(r"未登記建物",text) and  re.search(r"特殊關係間之交易",text) and not re.search(r"公共設施",text) and  (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "特殊關係交易及違章建物"
    
    elif isinstance(text,str) and not re.search(r"未登記建物",text) and  re.search(r"特殊關係間之交易",text) and re.search(r"公共設施",text) and not (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "特殊關係交易及公共設施"
    
    elif isinstance(text,str) and  re.search(r"未登記建物",text) and not re.search(r"特殊關係間之交易",text) and not re.search(r"公共設施",text) and  (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "未登記建物及違章建物"
    
    elif isinstance(text,str) and  re.search(r"未登記建物",text) and not re.search(r"特殊關係間之交易",text) and  re.search(r"公共設施",text) and not (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "未登記建物及公共設施"
    
    elif isinstance(text,str) and re.search(r"未登記建物",text) and  re.search(r"特殊關係間之交易",text) and not re.search(r"公共設施",text) and not (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "未登記建物及特殊關係交易"
    
    elif isinstance(text,str) and re.search(r"未登記建物",text) :
        return "未登記建物"
    
    elif isinstance(text,str) and re.search(r"公共設施",text):
        return "公共設施"
    
    elif isinstance(text,str) and (re.search(r"頂樓加蓋", text) or re.search(r'露臺外推',text) or re.search(r"陽台外推", text) or re.search(r"增建",text)):
        return "違章建物"
    
    elif isinstance(text,str) and re.search(r"特殊關係間之交易",text):
        
        return "特殊關係交易"
    
    elif isinstance(text,str) and re.search(r"政府機關標讓售",text):
        
        return "政府機關標售"
    
    else : 
        
        return text
    
    



In [379]:
data113.head()

,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,總樓層數,建物型態,主要用途,主要建材,建築完成年月,建物移轉總面積平方公尺,建物現況格局.房,建物現況格局.廳,建物現況格局.衛,建物現況格局.隔間,有無管理組織,總價元,單價元平方公尺,車位類別,車位移轉總面積平方公尺,車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號,m,n,經度,緯度,單價(萬/坪),土地移轉總面積(坪),建物移轉總面積(坪),車位移轉總面積(坪),主建物面積(坪),附屬建物面積(坪),陽台面積(坪),車位總價元(萬),總價元(萬),屋齡,是否為預購屋,第一筆樓層數,第二筆層數,是否夠買一層以上的房子,購買是否有買夾層屋,公設面積包含電梯或樓梯,公設面積包含屋頂突出物,附屬面積含平台,附屬面積含露台,購買時是否有騎樓,是否有購地下樓層,主建物占比,陽台面積占比,附屬面積占比,公佔比,是否有記附屬面積
0,士林區,車位,臺北市士林區芝玉路一段２２２、２２６、２２６之１號地下一層,4.99,住,NaN,NaN,2024-03-01,土地1建物1車位1,地下一層,18.0,其他,停車用途,鋼骨鋼筋混凝土造,2008-11-12,17.32,0,0,0,無,有,3300000,NaN,坡道平面,17.31,3300000,僅車位交易；,RPPNMLQJLHJGFEA67EA,0.00,0.00,0.00,有,2.0,113,3,25.109462,121.534093,NaN,1.509834,5.240545,5.237519,0.000000,0.000000,0.000000,330.0,330.0,15.309589,0,-1,,0,0,0,0,0,0,0,1,0.000000,0.000000,0.000000,1.000000e+00,0
1,士林區,車位,臺北市士林區中山北路七段３８巷１１號地下三層,0.53,住,NaN,NaN,2024-02-19,土地1建物1車位1,地下三層,12.0,其他,停車用途,鋼筋混凝土造,2008-09-04,44.93,0,0,0,無,有,3150000,NaN,坡道平面,44.93,3150000,僅車位交易；,RPPOMLLLKHJGFEA27DA,0.00,0.00,0.00,有,38.0,113,2,25.106497,121.527479,NaN,0.160363,13.594554,13.594554,0.000000,0.000000,0.000000,315.0,315.0,15.468493,0,-3,,0,0,0,0,0,0,0,1,0.000000,0.000000,0.000000,1.000000e+00,0
2,士林區,房地(土地+建物),臺北市士林區延平北路九段２１３號三樓,42.67,都市：其他:遊樂區(細部計畫尚未完成，尚未能准許依變更後計畫用途使用)。,NaN,NaN,2024-02-06,土地1建物1車位0,三層，陽臺,3.0,公寓(5樓含以下無電梯),住家用,鋼筋混凝土造,1976-02-14,71.63,6,1,1,有,無,10000000,139606.0,NaN,0.00,0,頂樓加蓋；,RPPOMLPJKHJGFEA47DA,71.63,0.00,0.00,無,NaN,113,2,25.098656,121.546989,46.139783,12.910741,21.673222,0.000000,21.673222,0.000000,0.000000,0.0,1000.0,48.010959,0,3,，陽臺,0,0,0,0,0,0,0,0,1.000000,0.000000,0.000000,0.000000e+00,1
3,士林區,房地(土地+建物)+車位,臺北市士林區天母西路５０巷２５號五樓,54.19,住,NaN,NaN,2024-02-15,土地1建物1車位2,五層,14.0,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,2001-04-23,304.02,4,2,3,有,有,68000000,223669.0,坡道平面,0.00,6000000,陽台外推；,RPRNMLOJLHJGFEA57DA,139.76,2.09,15.91,有,NaN,113,2,25.092872,121.506457,73.922605,16.396369,91.987897,0.000000,42.287443,0.632375,4.813918,600.0,6800.0,22.830137,0,5,,0,0,0,0,0,0,0,0,0.459707,0.052332,0.006875,4.810868e-01,1
4,士林區,房地(土地+建物),臺北市士林區社中街１２０號,18.00,住,NaN,NaN,2024-02-17,土地1建物1車位0,一層,4.0,公寓(5樓含以下無電梯),住商用,鋼筋混凝土造,1977-05-27,48.21,0,0,0,無,無,4490000,93134.0,NaN,0.00,0,親友、員工、共有人或其他特殊關係間之交易；,RPRNMLPJLHJGFEA67DA,44.27,3.94,0.00,無,NaN,113,2,25.137294,121.547139,30.780787,5.446293,14.586989,0.000000,13.394856,1.192133,0.000000,0.0,449.0,46.758904,0,1,,0,0,0,0,0,0,0,0,0.918274,0.000000,0.081726,9.714451e-17,1


In [380]:
data113['備註'] = data113['備註'].apply(lambda x : transforNOTE(x) )

In [381]:
print(data113['備註'].value_counts())

備註
違章建物                                                                                                                                                                                 5562
一般交易                                                                                                                                                                                 3860
特殊關係交易                                                                                                                                                                               1397
僅車位交易；                                                                                                                                                                                541
特殊關係交易及違章建物                                                                                                                                                                           244
夾層；                                                                

In [382]:
print(data113.head())

  鄉鎮市區          交易標的                       土地位置建物門牌  土地移轉總面積平方公尺  \
0  士林區            車位  臺北市士林區芝玉路一段２２２、２２６、２２６之１號地下一層         4.99   
1  士林區            車位         臺北市士林區中山北路七段３８巷１１號地下三層         0.53   
2  士林區     房地(土地+建物)             臺北市士林區延平北路九段２１３號三樓        42.67   
3  士林區  房地(土地+建物)+車位             臺北市士林區天母西路５０巷２５號五樓        54.19   
4  士林區     房地(土地+建物)                  臺北市士林區社中街１２０號        18.00   

                               都市土地使用分區  非都市土地使用分區  非都市土地使用編定      交易年月日  \
0                                     住        NaN        NaN 2024-03-01   
1                                     住        NaN        NaN 2024-02-19   
2  都市：其他:遊樂區(細部計畫尚未完成，尚未能准許依變更後計畫用途使用)。        NaN        NaN 2024-02-06   
3                                     住        NaN        NaN 2024-02-15   
4                                     住        NaN        NaN 2024-02-17   

       交易筆棟數   移轉層次  總樓層數             建物型態  主要用途      主要建材     建築完成年月  \
0  土地1建物1車位1   地下一層  18.0               其他  停車用途  鋼骨鋼筋混凝土造 20

In [383]:
new_data = data113[['鄉鎮市區', '交易標的', '土地位置建物門牌', '土地移轉總面積(坪)', '都市土地使用分區',

           '交易年月日', '交易筆棟數', '建物型態', '主要用途', '主要建材',

        '建物現況格局.房', '建物現況格局.廳', '建物現況格局.衛','建物現況格局.隔間', '有無管理組織',

          '車位類別', '電梯', 'm', 'n','單價(萬/坪)', '土地移轉總面積(坪)',

        '建物移轉總面積(坪)', '車位移轉總面積(坪)', '車位總價元(萬)','總價元(萬)', '主建物面積(坪)',

        '附屬建物面積(坪)', '陽台面積(坪)', '屋齡', '是否為預購屋','是否夠買一層以上的房子', '購買是否有買夾層屋',

        '公設面積包含電梯或樓梯', '公設面積包含屋頂突出物', '附屬面積含平台','附屬面積含露台', '購買時是否有騎樓', 

        '是否有購地下樓層', '主建物占比', '陽台面積占比','附屬面積占比', '公佔比', '是否有記附屬面積','備註','第一筆樓層數','第二筆層數']]



In [384]:
new_data.to_excel('C:\資料探勘\資料處理正式\complete_processing_data_113.xlsx')

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\ruizhong\AppData\Local\Temp\ipykernel_31040\3984877758.py:1: SyntaxWarning: invalid escape sequence '\c'
  new_data.to_excel('C:\資料探勘\資料處理正式\complete_processing_data_113.xlsx')
